<a href="https://colab.research.google.com/github/jyeongvv/DL_study/blob/main/jyeongvv/0403_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5_EDA_Baseline_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전이학습
* 이미지, 음성, 자연어 처리 등에서 매우 유용한 기술 중 하나
* 미리 학습된 모델의 지식을 가져와 새로운 문제에 적용하는 기술 -> 즉, 이미 학습된 모델의 일부 또는 전체 파라미터를 재사용하여 새로운 문제를 해결
  > 사전학습된 모델의 일부를 재사용하는 경우

  > 사전학습된 모델의 모든 레이어를 재사용하는 경우
  
  > 여러 개의 사전학습된 모델을 조합하여 새로운 모델을 생성하는 경우

# EDA

## 데이터 시각화
---

### 타깃값 분포

**데이터를 타깃값별로 추출**

* `train.loc[train['healthy'] == 1]` -> 예시

```
# [col for col in train.columns[1:]]
healthy, multiple_diseases, rust, scab = [
    train.loc[train[col] == 1] for col in train.columns[1:]
]
```



### 이미지 출력

* `assert len(img_ids) <= rows * cols` -> 이미지가 행/열 개수보다 많으면 오류 발생
  > assert는 True면은 문제가 없고, False일 때 에러를 발생시킴
* `image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)` 를 해주지 않으면 **BGR -> RGB**로 나옴
* [-num_of_lmgs:] -> 뒤에서 num_of_lmgs만큼의 갯수만큼 이미지 id를 추출

---

# Baseline

## 훈련데이터, 검증데이터 분리
* test_size=0.1, -> 10%를 검증셋으로 사용


## 데이터셋 클래스
*  데이터셋을 훈련용 또는 검증용 : **False** -> **라벨을 같이** 반환
* 테스트용으로 만들 거면 : **True** -> **이미지만** 반환
* `is_test = False,`

## 이미지 변환기
* 밝기 대비 조절 (+- 랜덤으로 밝기 비율 조정)
  * `A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3)`
* 상하 대칭 변환
  * `A.VerticalFlip(p=0.2)`
* 좌우 대칭 변환 
  * `A.HorizontalFlip(p=0.5)`
* 이동, 스케일링, 회전 변환
  * `A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.3)`
* 양각화 (튀어나오게), 날카로움, 블러 효과
  * `A.OneOf([A.Emboss(p=1), A.Sharpen(p=1), A.Blur(p=1),], p=0.3)`
* 어파인 변환 -> 이동, 확대/축소, 회전 => 이미지 모양을 전체적으로 바꾸는 변환기
  * `A.PiecewiseAffine(p=0.3)`
* 정규화 변환
  * `A.Normalize()`
* 텐서로 변환
  * `ToTensorV2`

### 데이터셋 생성
* 멀티 프로세싱


### 데이터로더 생성
* loader_train / loader_valid

## 모델 생성

* 전에는 계층(레이어)별로 구성
* 이거는 사전 훈련된 모델을 전이 학습시키는 것

* **사전 훈련 모델** : 이미 한 분야에서 훈련을 마친 모델
* **전이 학습** : 사전 훈련 모델을 유사한 다른 영역에 재훈련 시키는 기법

* **파이토치에서 사전 훈련 모델 사용하는 법**
1. torchvision.models 모듈 이용
* 제공하는 모델 ↓
* 기본적으로 설치되어 있기에 사용이 간단함

2. pretrainedmodels 모듈 이용

3. 직접 구현한 모듈 이용

### EfficientNet 모델 생성

* 사전 훈련된 efficientnet-b7 모델 불러오기
  * model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4)
* num_classes : 출력값의 개수

## 모델 훈련

* 옵티마이저 : `Adam -> AdamW` : Adam + 가중치 감쇠 추가로 적용
* 가중치 감쇠 `weight decay` : 가중치를 작게 조절하는 규제 기법으로, 과적합(과대적합)을 억제해줌
* 옵티마이저 내 기울기 초기화 -> `optimizer.zero_grad()`
* 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
  * `outputs = model(images)`
* 손실 함수를 활용해 outputs와 labels의 손실값 계산
  * `loss = criterion(outputs, labels)`
* 훈련 데이터 손실값 출력
  * `print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')`

## 예측 및 결과

* 예측값을 해당 id에 맞게 집어넣어주는 코드
  * `submission[submission.columns[1:]] = preds`